##### Copyright 2019 The TensorFlow Authors. [Licensed under the Apache License, Version 2.0](#scrollTo=Afd8bu4xJOgh).

In [ ]:
// #@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
// https://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/swift/tutorials/protocol_oriented_generics"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/swift/blob/master/docs/site/tutorials/protocol_oriented_generics.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/swift/blob/master/docs/site/tutorials/protocol_oriented_generics.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
</table>

# プロトコル指向プログラミングとジェネリック

このチュートリアルでは、プロトコル指向プログラミングを説明し、日常的な設定におけるジェネックとの使用方法をさまざまな例を通じて紹介します。

## プロトコル

プログラミング言語において、継承は、強力なコードの編成方法であり、プログラムの複数のコンポーネント間でのコードの共有を実現することができます。

Swift では、さまざまな方法で継承を表現できます。クラスの継承など、すでにいくつかの方法はほかの言語からご存じかもしれませんが、Swift ではプロトコルも継承可能です。

このチュートリアルでは、さまざまなトレードオフを通じて似たような目標を達成できる、サブクラス化の代替手法としてのプロトコルについて説明します。Swift では、プロトコルには複数の抽象メンバーが含まれます。クラス、構造、および列挙は複数のプロトコルに適合し、遡及的に適合関係を確立することができます。Swift においてサブクラス化では簡単に表現できない設計も、こういったプロトコルの仕様によって可能となります。プロトコルの使用をサポートするイディオム（拡張機能とプロトコルの制約）のほか、プロトコルの制限についても説明しています。


## Swift 💖 の値の型について

参照セマンティクスのあるクラスのほか、Swift は値として渡される列挙や構造をサポートしています。列挙と構造はクラスが提供する多数の機能をサポートしています。では、それらを見てみましょう！

まず、列挙とクラスがどれくらい似ているのかを確認しましょう。

In [ ]:
enum Color: String {
    case red = "red"
    case green = "green"
    case blue = "blue"
    // A computed property. Note that enums cannot contain stored properties.
    var hint: String {
        switch self {
            case .red:
                return "Roses are this color."
            case .green:
                return "Grass is this color."
            case .blue:
                return "The ocean is this color."
        }
    }
    
    // An initializer like for classes.
    init?(color: String) {
        switch color {
        case "red":
            self = .red
        case "green":
            self = .green
        case "blue":
            self = .blue
        default:
            return nil
        }
    }
}

// Can extend the enum as well!
extension Color {
    // A function.
    func hintFunc() -> String {
        return self.hint
    }
}

let c = Color.red
print("Give me a hint for c: \(c.hintFunc())")

let invalidColor = Color(color: "orange")
print("is invalidColor nil: \(invalidColor == nil)")

次に、構造を確認しましょう。構造を継承することはできない代わりにプロトコルを使用するところに注意してください。

In [ ]:
struct FastCar {
    // Can have variables and constants as stored properties.
    var color: Color
    let horsePower: Int
    // Can have computed properties.
    var watts: Float {
       return Float(horsePower) * 745.7
    }
    // Can have lazy variables like in classes!
    lazy var titleCaseColorString: String = {
        let colorString = color.rawValue
        return colorString.prefix(1).uppercased() + 
               colorString.lowercased().dropFirst()
    }()
    // A function.
    func description() -> String {
        return "This is a \(color) car with \(horsePower) horse power!"
    }
    // Can create a variety of initializers.
    init(color: Color, horsePower: Int) {
        self.color = color
        self.horsePower = horsePower
    }
    // Can define extra initializers other than the default one.
    init?(color: String, horsePower: Int) {
        guard let enumColor = Color(color: color) else {
            return nil
        }
        self.color = enumColor
        self.horsePower = horsePower
    }
}

var car = FastCar(color: .red, horsePower: 250)
print(car.description())
print("Horse power in watts: \(car.watts)")
print(car.titleCaseColorString)

最後に、列挙と構造が、クラスとは異なり、値の型として渡される様子を見てみましょう。

In [ ]:
// Notice we have no problem modifying a constant class with 
// variable properties.
class A {
  var a = "a"
}

func foo(_ a: A) {
  a.a = "foo"
}
let a = A()
print(a.a)
foo(a)
print(a.a)

/* 
Uncomment the following code to see how an error is thrown.
Structs are implicitly passed by value, so we cannot modify it.
> "error: cannot assign to property: 'car' is a 'let' constant"
*/

// func modify(car: FastCar, toColor color: Color) -> Void {
//   car.color = color
// }

// car = FastCar(color: .red, horsePower: 250)
// print(car.description())
// modify(car: &car, toColor: .blue)
// print(car.description())


## プロトコルを使いましょう

様々な車種のプロトコルを作成することから始めます。

In [ ]:
protocol Car {
    var color: Color { get set }
    var price: Int { get }
    func turnOn()
    mutating func drive()
}

protocol Electric {
    mutating func recharge()
    // percentage of the battery level, 0-100%.
    var batteryLevel: Int { get set }
}

protocol Gas {
    mutating func refill()
    // # of liters the car is holding, varies b/w models.
    var gasLevelLiters: Int { get set }
}

オブジェクト指向の（複数の継承を行えない）世界では、`Electric` と `Gas` という抽象クラスを作成してから、クラス継承を使用して両方を `Car` の継承とし、さらに特定の車種をベースクラスとしていたかもしれません。しかし、ここでは、これらはカップリングが**全くない**完全に独立したプロトコルになっています。このため、システム全体の設計方法にさらなる柔軟性が付加されています。

Tesla を定義してみましょう。

In [ ]:
struct TeslaModelS: Car, Electric {
    var color: Color // Needs to be a var since `Car` has a getter and setter.
    let price: Int
    var batteryLevel: Int
    
    func turnOn() {
        print("Starting all systems!")
    }

    mutating func drive() {
        print("Self driving engaged!")
        batteryLevel -= 8
    }

    mutating func recharge() {
        print("Recharging the battery...")
        batteryLevel = 100
    }
}

var tesla = TeslaModelS(color: .red, price: 110000, batteryLevel: 100)

これは、`Car` と `Electric` の両方のプロトコルに適合する新しい構造 `TeslaModelS` を指定しています。

では、ガソリン車を定義しましょう。

In [ ]:
struct Mustang: Car, Gas{
    var color: Color
    let price: Int
    var gasLevelLiters: Int
    
    func turnOn() {
        print("Starting all systems!")
    }
    
    mutating func drive() {
        print("Time to drive!")
        gasLevelLiters -= 1
    }
    
    mutating func refill() {
        print("Filling the tank...")
        gasLevelLiters = 25
    }
}

var mustang = Mustang(color: .red, price: 30000, gasLevelLiters: 25)

### デフォルトの動作を設定してプロトコルを拡張する

この例から、冗長性があることがわかります。電気自動車を充電するたびに、充電率のレベルを 100 に設定しなければなりません。ガソリン車の場合はガソリンタンクの容量によって異なりますが、すべての電気自動車の最大充電量は 100% であるため、電気自動車のデフォルトのレベルを 100 とすることができます。

ここで役立つのが Swift の拡張機能です。

In [ ]:
extension Electric {
    mutating func recharge() {
        print("Recharging the battery...")
        batteryLevel = 100
    }
}

こうすることで、以降で作成するすべての電気自動車は、充電する際にバッテリーを 100 に設定するようになりました。クラス構造、および列挙を、一意のデフォルトの動作でデコレートすることができました。

![Protocol Comic](https://koenig-media.raywenderlich.com/uploads/2015/06/protocols-extend.png)

この挿絵を描いてくれた [Ray Wenderlich](https://www.raywenderlich.com/814-introducing-protocol-oriented-programming-in-swift-3) に感謝！

しかし、気を付けなければならないことが 1 つあります。最初の実装では、`foo()` を `A` のデフォルトの実装として定義していますが、プロトコルでは必須になっていません。そのため、`a.foo()` を呼び出すと、「`A default`」が出力されてしまいます。

In [ ]:
protocol Default {}

extension Default {
    func foo() { print("A default")}
}

struct DefaultStruct: Default {
    func foo() {
        print("Inst")
    }
}

let a: Default = DefaultStruct()
a.foo()

しかし、`foo()` を `A` で必須にすると、「`Inst`」が出力されます。

In [ ]:
protocol Default {
    func foo()
}

extension Default {
    func foo() { 
        print("A default")
    }
}

struct DefaultStruct: Default {
    func foo() {
        print("Inst")
    }
}

let a: Default = DefaultStruct()
a.foo()

これは、Swift のプロトコルにおける最初の例の静的ディスパッチと 2 つ目の性的ディスパッチが異なるためです。詳細については、こちらの[Medium の記事](https://medium.com/@PavloShadov/https-medium-com-pavloshadov-swift-protocols-magic-of-dynamic-static-methods-dispatches-dfe0e0c85509)をご覧ください。

### デフォルトの動作をオーバーライドする

それでも、必要であれば、デフォルトの動作をオーバーライドすることができます。これは[動的ディスパッチはサポートしていない](https://stackoverflow.com/questions/44703205/swift-protocol-extension-method-is-called-instead-of-method-implemented-in-subcl)ことに注意してください。

より古いバージョンの電気自動車があり、そのためバッテリーヘルスが 90% に減少してしまっていると仮定しましょう。

In [ ]:
struct OldElectric: Car, Electric {
    var color: Color // Needs to be a var since `Car` has a getter and setter.
    let price: Int
    var batteryLevel: Int
    
    func turnOn() {
        print("Starting all systems!")
    }
    
    mutating func drive() {
        print("Self driving engaged!")
        batteryLevel -= 8
    }
    
    mutating func reCharge() {
        print("Recharging the battery...")
        batteryLevel = 90
    }
}

## プロトコルの標準ライブラリ使用

Swift でどのようにプロトコルが機能するかを理解したので、標準ライブラリプロトコルを使用する典型的な例を見てみましょう。

### 標準ライブラリを拡張する

すでに Swift に存在する型に機能を追加する方法を確認します。Swift の型はビルトインではなく構造として標準ライブラリの一部であるため、機能の追加は簡単です。

配列の要素に対して二分探索を行いながら、配列がソートされていることを確認しましょう。

In [ ]:
extension Collection where Element: Comparable {
    // Verify that a `Collection` is sorted.
    func isSorted(_ order: (Element, Element) -> Bool) -> Bool {
        var i = index(startIndex, offsetBy: 1)
        
        while i < endIndex {
            // The longer way of calling a binary function like `<(_:_:)`, 
            // `<=(_:_:)`, `==(_:_:)`, etc.
            guard order(self[index(i, offsetBy: -1)], self[i]) else {
                return false
            }
            i = index(after: i)
        }
        return true
    }
    
    // Perform binary search on a `Collection`, verifying it is sorted.
    func binarySearch(_ element: Element) -> Index? {
        guard self.isSorted(<=) else {
            return nil
        }
        
        var low = startIndex
        var high = endIndex
        
        while low <= high {
            let mid = index(low, offsetBy: distance(from: low, to: high)/2)

            if self[mid] == element {
                return mid
            } else if self[mid] < element {
                low = index(after: mid)
            } else {
                high = index(mid, offsetBy: -1)
            }
        }
        
        return nil
    }
}

print([2, 2, 5, 7, 11, 13, 17].binarySearch(5)!)
print(["a", "b", "c", "d"].binarySearch("b")!)
print([1.1, 2.2, 3.3, 4.4, 5.5].binarySearch(3.3)!)

これは、*「要素を非破壊的に何度もトラバースし、インデックス付きのサブスクリプトによってアクセスされるシーケンス」*を定義する [`Collection`](https://developer.apple.com/documentation/swift/collection) プロトコルを拡張して行います。配列は角括弧表記を使ってインデックス化できるため、これが拡張するプロトコルです。

同様に、比較可能な要素を持つ配列にのみ、このユーティリティ関数を追加します。`where Element: Comparable` はこのために存在します。

`where` 句は、Swift の型体系の一部です。これは後で説明しますが、手短に述べると、記述している拡張機能に、型にプロトコルを実装しなければならないという要件、2 つの型は同一でなければならないという要件、またはクラスに特定のスーパークラスが必要であるという要件などを追加することができます。

[`Element`](https://developer.apple.com/documentation/swift/sequence/2908099-element) は、`Collection` 準拠型の要素の関連型です。`Element` は、[`Sequence`](https://developer.apple.com/documentation/swift/sequence) プロトコル内に定義されていますが、`Collection` は `Sequence` から継承するため、`Element` 関連型を継承します。

[`Comparable`](https://developer.apple.com/documentation/swift/comparable) は、*「リレーショナル演算子 `<`、`<=`、`>=`、および `>` を使って比較できる型」*を定義するプロトコルです。ソート済みの `Collection` に対して二分探索を実行しているため、これはもちろん true である必要があります。true でない場合は、二分探索で左から右に再帰/イテレートするのかがわかりません。

実装の注意書きとして、使用された `index(_:offsetBy:)` 関数についての詳細は、次の[ドキュメント](https://developer.apple.com/documentation/swift/string/1786175-index)をご覧ください。

## ジェネリック + プロトコル = 💥

ジェネリックとプロトコルは、コードを重複させないように正しく使用されれば強力なツールと言えます。

まず、[Swift ツアー](https://colab.research.google.com/github/tensorflow/swift/blob/master/docs/site/tutorials/a_swift_tour.ipynb)という別のチュートリアルをご覧ください。Colab ブックの最後に、ジェネリックについて簡単に説明されています。

ジェネリックがどのようなものか大まかに理解していると仮定し、少し高度な使用方法をさっと見てみましょう。

単一の型に、複数のプロトコルに適合する型といった要件が複数ある場合、いくつかのオプションを使用できます。

In [ ]:
typealias ComparableReal = Comparable & FloatingPoint

func foo1<T: ComparableReal>(a: T, b: T) -> Bool {
    return a > b
}

func foo2<T: Comparable & FloatingPoint>(a: T, b: T) -> Bool {
    return a > b
}

func foo3<T>(a: T, b: T) -> Bool where T: ComparableReal {
    return a > b
}

func foo4<T>(a: T, b: T) -> Bool where T: Comparable & FloatingPoint {
    return a > b
}

func foo5<T: FloatingPoint>(a: T, b: T) -> Bool where T: Comparable {
    return a > b
}

print(foo1(a: 1, b: 2))
print(foo2(a: 1, b: 2))
print(foo3(a: 1, b: 2))
print(foo4(a: 1, b: 2))
print(foo5(a: 1, b: 2))

一番上で `typealias` が使用されいるところに注目してください。これは、既存の型の名前付きエイリアスをプログラムに追加します。型エイリアスが宣言された後は、既存の型の代わりにエイリアス名をプログラム中で使用できるようになります。型エイリアスは、新しい型を作成するのではなく、既存の型を名前で参照できるようにするだけです。

次に、プロトコルとジェネリックを合わせて使用する方法を見てみましょう。

コンピュータストアを運営しており、販売するノートブックについて、在庫をどのように整理するかを決定する上で、次の要件があるとします。

In [ ]:
enum Box {
    case small
    case medium
    case large
}

enum Mass {
    case light
    case medium
    case heavy
}

// Note: `CustomStringConvertible` protocol lets us pretty-print a `Laptop`.
struct Laptop: CustomStringConvertible {
    var name: String
    var box: Box
    var mass: Mass
    
    var description: String {
        return "(\(self.name) \(self.box) \(self.mass))"
    }
}

しかし、棚には重量制限があるため、`Laptop` を質量別にグループ分けするという新しい要件が加わりました。

In [ ]:
func filtering(_ laptops: [Laptop], by mass: Mass) -> [Laptop] {
    return laptops.filter { $0.mass == mass }
}

let laptops: [Laptop] = [
    Laptop(name: "a", box: .small, mass: .light),
    Laptop(name: "b", box: .large, mass: .medium),
    Laptop(name: "c", box: .medium, mass: .heavy),
    Laptop(name: "d", box: .large, mass: .light)
]

let filteredLaptops = filtering(laptops, by: .light)
print(filteredLaptops)

さらに、`Mass` 以外の要素で絞り込む場合はどうすればよいのでしょうか。

1 つのオプションとして、次のように行うことができます。

In [ ]:
// Define a protocol which will act as our comparator.
protocol DeviceFilterPredicate {
    associatedtype Device
    func shouldKeep(_ item: Device) -> Bool
}

// Define the structs we will use for passing into our filtering function.
struct BoxFilter: DeviceFilterPredicate {
    typealias Device = Laptop
    var box: Box 
    
    func shouldKeep(_ item: Laptop) -> Bool {
        return item.box == box
    }
}

struct MassFilter: DeviceFilterPredicate {
    typealias Device = Laptop  
    var mass: Mass
    
    func shouldKeep(_ item: Laptop) -> Bool {
        return item.mass == mass
    }
}

// Make sure our filter conforms to `DeviceFilterPredicate` and that we are 
// filtering `Laptop`s.
func filtering<F: DeviceFilterPredicate>(
    _ laptops: [Laptop], 
    by filter: F
) -> [Laptop] where Laptop == F.Device {
    return laptops.filter { filter.shouldKeep($0) }
}

// Let's test the function out!
print(filtering(laptops, by: BoxFilter(box: .large)))
print(filtering(laptops, by: MassFilter(mass: .heavy)))

素晴らしい！これで、ノートブックの任意の制約に基づいてフィルタできるようになりました。ただし、フィルタできるのは `Laptop` のみです。

箱に含まれており質量のあるものでフィルタリングするようにできないでしょうか。このノートブックの倉庫は、さまざまな顧客ベースのあるサーバーにも使用されているかもしれません。

In [ ]:
// Define 2 new protocols so we can filter anything in a box and which has mass.
protocol Weighable {
    var mass: Mass { get }
}

protocol Boxed {
    var box: Box { get }
}

// Define the new Laptop and Server struct which have mass and a box.
struct Laptop: CustomStringConvertible, Boxed, Weighable {
    var name: String
    var box: Box
    var mass: Mass
    
    var description: String {
        return "(\(self.name) \(self.box) \(self.mass))"
    }
}

struct Server: CustomStringConvertible, Boxed, Weighable {
    var isWorking: Bool
    var name: String
    let box: Box
    let mass: Mass

    var description: String {
        if isWorking {
            return "(working \(self.name) \(self.box) \(self.mass))"
        } else {
            return "(notWorking \(self.name) \(self.box) \(self.mass))"
        }
    }
}

// Define the structs we will use for passing into our filtering function.
struct BoxFilter<T: Boxed>: DeviceFilterPredicate {
    var box: Box 
  
    func shouldKeep(_ item: T) -> Bool {
        return item.box == box
    }
}

struct MassFilter<T: Weighable>: DeviceFilterPredicate {
    var mass: Mass
    
    func shouldKeep(_ item: T) -> Bool {
        return item.mass == mass
    }
}

// Define the new filter function.
func filtering<F: DeviceFilterPredicate, T>(
    _ elements: [T], 
    by filter: F
) -> [T] where T == F.Device {
    return elements.filter { filter.shouldKeep($0) }
}


// Let's test the function out!
let servers = [
    Server(isWorking: true, name: "serverA", box: .small, mass: .heavy),
    Server(isWorking: false, name: "serverB", box: .medium, mass: .medium),
    Server(isWorking: true, name: "serverC", box: .large, mass: .light),
    Server(isWorking: false, name: "serverD", box: .medium, mass: .light),
    Server(isWorking: true, name: "serverE", box: .small, mass: .heavy)
]

let products = [
    Laptop(name: "a", box: .small, mass: .light),
    Laptop(name: "b", box: .large, mass: .medium),
    Laptop(name: "c", box: .medium, mass: .heavy),
    Laptop(name: "d", box: .large, mass: .light)
]

print(filtering(servers, by: BoxFilter(box: .small)))
print(filtering(servers, by: MassFilter(mass: .medium)))

print(filtering(products, by: BoxFilter(box: .small)))
print(filtering(products, by: MassFilter(mass: .medium)))

これで、特定の `struct` のすべてのプロパティでだけでなく、そのプロパティを持つすべての構造体でもフィルタできるようになりました。

# 最適な API 設計を得るためのヒント

***このセクションは、[WWDC 2019: Modern Swift API Design](https://developer.apple.com/videos/play/wwdc2019/415/) の講演から抜粋したものです。***

プロトコルの振る舞いを理解したので、プロトコルを使用するタイミングについて説明することにします。プロトコルが強力だからといって、すぐにプロトコルを使用し始めるのは必ずしも最善策というわけではありません。

- 具体的なユースケースから始めましょう。
    - まずは具体的な型が伴うユースケースを調査し、共有して検索するどのコードが繰り返されているのかを把握します。次に、その共有コードをジェネリックで抜き取ります。新しいプロトコルを作成した方が良いのかもしれません。ジェネリックコードの必要性を探索してみましょう。
- 標準ライブラリに定義されている既存のプロトコルから新しいプロトコルを作成することを検討しましょう。この良い例につちえは、次の [Apple ドキュメント](https://developer.apple.com/documentation/swift/adopting_common_protocols)をご覧ください。
- ジェネリックプロトコルの代わりに、ジェネリック型を定義することを検討しましょう。

## 例: カスタムベクトル型を定義する

3 つの重要なベクトル演算を定義する、作成中の幾何学アプリで使用するために、浮動小数点数で `GeometricVector` プロトコルを定義するとします。

```swift
protocol GeometricVector {
    associatedtype Scalar: FloatingPoint
    static func dot(_ a: Self, _ b: Self) -&gt; Scalar
    var length: Scalar { get }
    func distance(to other: Self) -&gt; Scalar
}
```

ベクトルの次元を保存するとします。これには `SIMD` プロトコルを利用できるため、新しい型で `SIMD` プロトコルを改善することにします。`SIMD` ベクトルは、ベクトル演算を実行するために使用する場合に非常に高速な、固定サイズのベクトルとして捉えることができます。

```swift
protocol GeometricVector: SIMD {
    associatedtype Scalar: FloatingPoint
    static func dot(_ a: Self, _ b: Self) -&gt; Scalar
    var length: Scalar { get }
    func distance(to other: Self) -&gt; Scalar
}
```

次に、上記の演算のデフォルトの実装を定義しましょう。

```swift
extension GeometricVector {
    static func dot(_ a: Self, _ b: Self) -&gt; Scalar {
        (a * b).sum()
    }

    var length: Scalar {
        Self.dot(self, self).squareRoot()
    }

    func distance(to other: Self) -&gt; Scalar {
        (self - other).length
    }
}
```

そして、これらの機能を追加する型に適合性を追加する必要があります。

```swift
extension SIMD2: GeometricVector where Scalar: FloatingPoint { } extension SIMD3: GeometricVector where Scalar: FloatingPoint { } extension SIMD4: GeometricVector where Scalar: FloatingPoint { } extension SIMD8: GeometricVector where Scalar: FloatingPoint { } extension SIMD16: GeometricVector where Scalar: FloatingPoint { } extension SIMD32: GeometricVector where Scalar: FloatingPoint { } extension SIMD64: GeometricVector where Scalar: FloatingPoint { }
```

デフォルトの実装を提供してから複数の型に適合性を追加するという、上記の 3 つのステップで行うプロトコルの定義プロセスは非常に多く繰り返されます。

## プロトコルは必要でしたか？

一意の実装を持つ `SIMD` 型がないというのは、警告サインとして捉えることができます。このケースでは、プロトコルは特に役に立っていないということです。

## `SIMD` の拡張機能で定義する

`SIMD` プロトコルの拡張機能に 3 つの演算子を記述すれば、問題をより簡潔に解決できます。

```swift
extension SIMD where Scalar: FloatingPoint {
    static func dot(_ a: Self, _ b: Self) -&gt; Scalar {
        (a * b).sum()
    }

    var length: Scalar {
        Self.dot(self, self).squareRoot()
    }

    func distance(to other: Self) -&gt; Scalar {
        (self - other).length
    }
}
```

コードの行数を減らし、`SIMD` のすべての型にデフォルト実装をすべて追加しました。

この型の階層を作成したくなることがあるかもしれませんが、必ずしも必要ではないことに留意してください。また、コンパイル済みのプログラムのバイナリサイズが縮小され、コンパイルの加速化にもつながるでしょう。

ただし、この拡張機能のアプローチは、追加するメソッドがいくつかある場合に最適ではありますが、より大規模な API を設計する場合にはスケーラビリティの問題が発生してしまいます。

## Is-a と Has-a

前の方で、`GeometricVector` は `SIMD` を改善すると述べました。しかし、これはIs-a 関係でしょうか。問題は、`SIMD` がスカラー 1 とベクトルに追加できる演算を定義しますが、幾何学のコンテキストでそのような演算を定義する意味がないということです。

そのため、`SIMD` を、あらゆる浮動小数点数を処理できる新しいジェネリック型でラップする Has-a 関係の方が適しているかもしれません。

```swift
// NOTE: `Storage` is the underlying type that is storing the values,
// just like in a `SIMD` vector.
struct GeometricVector<storage: simd> where Storage.Scalar: FloatingPoint {
    typealias Scalar = Storage.Scalar
    var value: Storage
    init(_ value: Storage) { self.value = value }
}</storage:>
```

そして、慎重に、幾何学のコンテキストでのみ意味のある演算のみを定義することができます。

```swift
extension GeometricVector {
    static func + (a: Self, b: Self) -&gt; Self {
        Self(a.value + b.value)
    }

    static func - (a: Self, b: Self) -&gt; Self {
        Self(a.value - b.value)
    }
    static func * (a: Self, b: Scalar) -&gt; Self {
        Self(a.value * b)
    }
}
```

それでも、ジェネリック拡張機能を使用して、前に実装しようとしていた 3 つの演算子を取得することができます。これは、前とほぼ全く同じです。

```swift
extension GeometricVector {
    static func dot(_ a: Self, _ b: Self) -&gt; Scalar {
        (a.value * b.value).sum()
    }

    var length: Scalar {
        Self.dot(self, self).squareRoot()
    }

    func distance(to other: Self) -&gt; Scalar {
        (self - other).length
    }
}
```

全体として振り返ると、単に構造を使用することで、3 つの演算の動作を 1 つの型に精緻化することができました。プロトコルでは、反復的な適合性をすべての `SIMD` ベクトルに記述しなければならない問題に遭遇し、`Scalar + Vector` といった特定の演算子を使用できないようにすることはできませんでした（この場合では、使用できないようにしませんでした）。したがって、プロトコルは究極のソリューションではなく、より従来的なソリューションがより強力であることもあることに留意しましょう。

# プロトコル指向プログラミングに関するその他のリソース

ここで取り扱ったトピックに関するその他のリソースを以下に示します。

- [WWDC 2015: Protocol-Oriented Programming in Swift](https://developer.apple.com/videos/play/wwdc2015/408/): これは Swift 2 を使って紹介されているため、多くの内容が現バージョンとは異なりますが（プレゼンテーションで使用されているプロトコル名など）、理論とそれに基づく使用方法を理解する上で役立つリソースです。
- [Introducing Protocol-Oriented Programming in Swift 3](https://www.raywenderlich.com/814-introducing-protocol-oriented-programming-in-swift-3): これは Swift 3 で書かれているため、正しくコンパイルするにはコードを変更しなければならい箇所があるかもしれませんが、素晴らしいリソースです。
- [WWDC 2019: Modern Swift API Design](https://developer.apple.com/videos/play/wwdc2019/415/): 値と参照型の違い、API 設計においてプロトコルが悪い選択肢であることを実証するユースケース（上記の「最適な API 設計を得るためのヒント」と同じことを言及しています）、キーパスのメンバールックアップ、、およびプロパティラッパーについて説明されています。
- [Generics](https://docs.swift.org/swift-book/LanguageGuide/Generics.html): Swift 5 のジェネリックについて説明された Swift のドキュメントです。